<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250615%5D%20FedAvg%20%E2%80%93%20WSN-DS%20%E2%80%93%2010%20%25%20sign-flip%20attack%20(client%202)%2075R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FedAvg – WSN-DS – 10 % sign-flip attack (client 2)**

In [1]:
#!/usr/bin/env python3
# FedAvg – WSN-DS – 10 % sign-flip attack (client 2)
import os, time, numpy as np, pandas as pd, tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [2]:
# ----- config -----
SEED, NUM_CLIENTS, ATTACKER_ID = 42, 5, 2
ROUNDS, LOCAL_EPOCHS, BATCH_SIZE = 75, 1, 32
DATA_PATH, LOG_DIR = "dataset_WSN-DS.csv", "/mnt/data"

np.random.seed(SEED); tf.random.set_seed(SEED)

In [3]:
# ----- data -----
df = pd.read_csv("dataset_WSN-DS.csv").dropna()
for col in df.select_dtypes(include="object"):
    df[col] = LabelEncoder().fit_transform(df[col])
X,y = df.drop("Class",axis=1).values.astype("float32"), df["Class"].values
X = StandardScaler().fit_transform(X)
X,y = SMOTE(random_state=SEED).fit_resample(X,y)
X_tr,X_te,y_tr,y_te = train_test_split(X,y,test_size=0.2,stratify=y,random_state=SEED)
client_X = np.array_split(X_tr,NUM_CLIENTS)
client_y = np.array_split(y_tr,NUM_CLIENTS)
X_te = X_te[...,None]; INPUT_SHAPE=(X_te.shape[1],1)
y_te_cat = to_categorical(y_te); NUM_CLASSES=y_te_cat.shape[1]

In [4]:
def build_model():
    m=Sequential([InputLayer(input_shape=INPUT_SHAPE),
                  LSTM(64,activation='tanh'),
                  Dense(128,activation='relu'),Dropout(0.30),
                  Dense(64,activation='relu'),Dropout(0.30),
                  Dense(NUM_CLASSES,activation='softmax')])
    m.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    return m

global_model=build_model(); g_weights=global_model.get_weights()
perf_log=[]

for rnd in range(1,ROUNDS+1):
    t0=time.time(); cl_wts,cl_sz=[],[]
    for cid,(Xi,yi) in enumerate(zip(client_X,client_y)):
        local=build_model(); local.set_weights(g_weights)
        local.fit(Xi[...,None],to_categorical(yi,NUM_CLASSES),
                  epochs=LOCAL_EPOCHS,batch_size=BATCH_SIZE,verbose=0)
        w = local.get_weights()
        # ---- attack ----
        if cid == ATTACKER_ID:
            w = [-layer for layer in w]
        cl_wts.append(w); cl_sz.append(len(yi))

    tot = sum(cl_sz)
    g_weights=[np.sum([(n/tot)*w[l] for w,n in zip(cl_wts,cl_sz)],axis=0)
               for l in range(len(g_weights))]
    global_model.set_weights(g_weights)

    y_pred=np.argmax(global_model.predict(X_te,verbose=0),axis=1)
    perf_log.append(dict(round=rnd,
                         accuracy=accuracy_score(y_te,y_pred),
                         precision=precision_score(y_te,y_pred,average='weighted',zero_division=0),
                         recall=recall_score(y_te,y_pred,average='weighted',zero_division=0),
                         f1=f1_score(y_te,y_pred,average='weighted',zero_division=0),
                         ms=round((time.time()-t0)*1000,2)))
    print(f"R{rnd:02d} acc={perf_log[-1]['accuracy']:.8f} "
          f"F1={perf_log[-1]['f1']:.8f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


R01 acc=0.43186029 F1=0.33827140


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


R02 acc=0.53746920 F1=0.45038302


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...

R73 acc=0.44678974 F1=0.34146266


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


R74 acc=0.44166721 F1=0.32970429


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


R75 acc=0.44736022 F1=0.33246270


In [5]:
# ----- save -----
os.makedirs(LOG_DIR,exist_ok=True)
pd.DataFrame(perf_log).to_csv(f"{LOG_DIR}/perf_log_DS_fedavg_poison.csv",index=False)
print("\n✓ FedAvg poisoned run complete")


✓ FedAvg poisoned run complete


In [6]:
# prompt: download perf_log_DS_fedavg_poison.csv file

from google.colab import files
files.download('/mnt/data/perf_log_DS_fedavg_poison.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>